![Unive logo](assets/logo_caFoscari.png "Università Ca'Foscari di Venezia")

# Analisi della sicurezza attraverso lo studio di indici

L’attività presentata consiste nell’analizzare il livello di sicurezza informatica di un ecosistema attraverso il calcolo e l’interpretazione di indici. Questi verranno suddivisi in tre categorie principali: di **Conformità**, di **Andamento** e di **Anomalia**.

 > ##### Conformità
 >
 >  Un indice di conformità rappresenta una **valutazione della conformità** dell’ambiente di studio rispetto ad un determinato target. 

 > ##### Andamento
 >
 >  Gli indici di andamento possono essere considerati come i parametri più significativi essendo di immediata comprensione. Rappresentano **l’andamento** di una particolare **attività** in un determinato periodo.

 > ##### Anomalia
 >
 >  Un indice di anomalia invece consiste in una valutazione appositamente creata per identificare anomalie. Si analizzano i **comportamenti** dei soggetti all’interno dell’ambiente di studio e si individuano potenziali **anomalie** rispetto ad un valore atteso. 

***

## Metodologia e approccio

In questa sezione verrà descritta la metodologia utilizzata per portare a termine l'attività. Questa può essere riassunta in tre fasi, qui di seguito è presente una descrizione dettagliata per ognuna di queste. 


##### 1.  Prima fase -  Tecnologie
    L’attività principale della prima fase è stata di studio delle tecnologie utilizzate. 
        - Microsoft Azure e Azure Portal
        - Log Analytics e Azure Monitor
        - Microsoft Sentinel
        
    Altre tecnologie sono state utilizzate indirettamente tra cui MITRE ATT&K framework e Jupyter/Databricks notebook.

##### 2.  Seconda fase - Analisi
    La seconda fase comprende uno studio preliminare alla teoria degli indici e sulla distinzione tra KPI e KRI.
    
    Successivamente è stata fatta un analisi dei dati per la selezione di quali indici generare.
    
    I dati analizzati provengono da ambienti Sentinel di dimostrazione (Demo) e da ambienti Sentinel gestiti da Horizon Security s.r.l.

##### 3.  Terza fase - Calcolo
    La terza ed ultima fase consiste nel calcolo degli indici precedentemente scelti e nella produzione di un grafico che permette una rappresentazione grafica dell’indice stesso.
    
    Ogni grafico è corredato da una serie di considerazioni sui risultati e da una serie di proposte per la correzione di eventuali problematiche.

***

## Tecnologie utilizzate

##### - Microsoft Azure
    Microsoft Azure è la piattaforma cloud pubblica di Microsoft, che offre servizi di cloud computing.
    
    Tramite Azure vengono erogati servizi appartenenti a diverse categorie quali:
        - risorse di elaborazione
        - archiviazione e memorizzazione dati
        - trasmissione dati e interconnessione di reti
        - analisi e apprendimento automatico
        - sicurezza e gestione delle identità
        
    I servizi messi a disposizione da Microsoft Azure possono essere classificati in tre aree, a seconda della modalità di erogazione adottata: Infrastructure as a Service (IaaS), Platform as a Service (PaaS) e infine Software as a Service (SaaS). 
   ![Matrice di responsabilità](/assets/mat_resp.png "Distribuzione delle responsabilità")
   
   
##### - Microsoft Sentinel
    I dati con i quali sono stati sviluppati gli indici provengono da Microsoft Sentinel.
    
    Microsoft Sentinel è una soluzione di orchestrazione, automazione e risposta ( SOAR) scalabile, nativa del cloud, per la gestione delle informazioni e degli eventi di sicurezza (SIEM, Security Orchestration, Automation, and Response).
    
    Offre analisi della sicurezza intelligente e intelligence per le minacce in tutta l'azienda, offrendo un'unica soluzione per il rilevamento degli attacchi, la visibilità delle minacce, la ricerca proattiva e la risposta alle minacce.

***

## Analisi dei dati

> ‹‹ Microsoft Sentinel è la visione d'occhio dell'azienda che attenua lo stress degli attacchi sempre più sofisticati ››

##### - Provenienza
    Sentinel raccoglie i dati su scala cloud per tutti gli utenti, i dispositivi, le applicazioni e l'infrastruttura, in locale e in più cloud.
    Microsoft Sentinel include una serie di connettori per le principali soluzioni Microsoft:
        - Office 365
        - Azure AD 
        - Microsoft Defender for Identity 
        - Microsoft Defender for Cloud Apps
        - Microsoft Defender for Endpoint 
        
    Inoltre, sono presenti connettori predefiniti a soluzioni non Microsoft, per l'ecosistema di sicurezza allargato. 
    È anche possibile usare il formato di evento comune, Syslog o REST-API per connettere le origini dati anche a Microsoft Sentinel.

   
   
##### - Struttura dei dati
    Microsoft Sentinel incorpora in modo nativo il servizio Log Analytics. Log Analytics è uno strumento che consente di eseguire query sui dati raccolti dai log di Monitoraggio di Azure e analizzarne i risultati in modo interattivo.
    
    Microsoft Sentinel arricchisce l'analisi e il rilevamento con l'intelligenza artificiale e fornisce il flusso di intelligence sulle minacce di Microsoft e consente di ottenere informazioni sulle minacce.
    
    I dati provenienti dall’analisi dei file di Log vengono memorizzati all’interno dell’ambiente Microsoft Sentinel sotto forma di tabelle.
    Questa base di dati è interrogabile attraverso un linguaggio di interrogazione chiamato Kusto (KQL).

    KQL funziona con uno schema organizzato tramite una gerarchia simile a quella di SQL: database, tabelle e colonne.
    Infatti KQL non è altro che un dialetto di SQL.

***

## Calcolo dei dati

In questa sezione vedremo quali sono gli indici scelti e ne vedremo il loro calcolo e una loro rappresentazione grafica.

### 1. Monitor sui tentativi di MFA falliti

Questo è un **indice di anomalia** e successivamente verrà mostrato un grafico contenente i tentativi di login falliti a causa della fallita autenticazione a due fattori (MFA) degli ultimi 5 giorni, raggruppati in base all’ora.

Si riesce a notare che il girono 5/03/2022 alle 00:00:00 c’è stato un insolito picco di tentativi che si sono conclusi con un fallimento dell’autenticazione a due fattori.

La query KQL relativa a questi dati è: 

```
SignInLogs
| where TimeGenerated > ago(5d)
| where ResultDescription has "not pass the MFA"
| summarize Count=count() by bin(TimeGenerated, 1h)
```

In [0]:
%sql
select Time, count(*) as Count 
from default.mfafailedaccess
where ResultDescription like "%not pass the MFA%"
group by Time
order by Time

##### Considerazioni
    Questo picco potrebbe essere causato da un tentativo di brute force del secondo fattore a seguito di un furto di credenziali da parte di un attaccante.
    
    Le possibili azioni correttive a valle di un indagine sono:
        - Blocco dell’account compromesso
        - Rifiuto di ulteriori tentativi di connessione da quell’indirizzo IP

***

### 2. Monitor sui tentativi di login

Anche questo è un **indice di anomalia** e verrà mostrato un grafico contenente il conteggio dei tentativi di login degli ultimi 3 giorni. Vengono poi raggruppati in base all’ora.

Questo grafico sarebbe del tutto in linea con un contesto lavorativo ma presenta un anomalia, c’è stato un insolito tentativo di accessi in un orario notturno (14/03/2022, 23:00:00).

La query KQL relativa a questi dati è:
   ```
   SignInLogs
   | where TimeGenerated > ago(3d)
   | summarize LoginCount=count() by bin(TimeGenerated, 1h)
   ```

In [0]:
%sql

CREATE VIEW count_login
AS( select TimeGenerated, count(TimeGenerated) as LoginCount
    from loginattempts
    group by TimeGenerated )


In [0]:
%sql

select Date, LoginCount
from plaindate left join count_login on plaindate.Date = count_login.TimeGenerated
where Date between DATE'2022-03-14' and DATE'2022-03-17'
order by Date

##### Considerazioni
    Questo potrebbe rappresentare un tentativo da parte di un attaccante di avere accesso ad un sistema tramite un attacco di brute force delle credenziali di accesso.

    Le possibili azioni correttive a valle di un indagine sono:
        - Aggiungere più fattori di autenticazione nelle ore notturne.
        - Aggiungere un limite massimo ai tentativi di login.


***

### 3. Monitor sulle attività di rete

Un ultimo **indice di anomalia** dove il seguente grafico analizza l’attività di rete effettiva (blu) e le paragona ad un’attività di rete standard (rosso). 


Si nota che il traffico di rete è elevato anche nelle ore notturne tra il 12 e il 13 e tra il 13 e il 14 marzo 2022. 

Le attività di rete rimangono sotto la soglia di detection (nessun picco troppo elevato) ma costanti anche durante la notte.

In [0]:
%sql

select act.TimeGenerated, act.Actual_Count, exp.Expected_Count 
from processes_actual_trend as act 
  inner join processes_expected_trend as exp on act.TimeGenerated = exp.TimeGenerated
order by act.TimeGenerated

##### Considerazioni
    In questo caso potrebbe esserci il rischio di esfiltrazione di informazioni sensibili. Un attaccante mantiene delle comunicazioni attive per esportare dati.

    Le possibili azioni correttive a valle di un indagine sono:
        - Bloccare le comunicazioni verso l’indirizzo/gli indirizzi IP
        - Implementare sistemi di encryption dei dati


***

### 4. Trend dei processi eseguiti

Questo primo **indice di andamento** studia e analizza il conteggio delle esecuzioni dei processi nell’arco di un mese.

Nella tabella SecurityEvent, presente in Microsoft Sentinel, l’identificatore dell’evento riguardante l’esecuzione di un processo è 4688.

La query KQL relativa a questi dati è:
   ```
   SecurityEvent
   | where TimeGenerated > ago(30d)
   | where EventID == 4688
   | summarize Process_Count=count() by bin(TimeGenerated, 1h)
   ```

In [0]:
%sql 

select Time as Date, Process_Count 
from processexecution
order by Time

##### Considerazioni
    Possiamo notare che il trend dei processi eseguiti è costante, non si presuppone l’esistenza di un attività sospetta. Non c’è necessità di ulteriori indagini.

    Nessuna azione correttiva è necessaria.


***

### 5. Trend dei password / name reset

Un ulteriore **indice di andamento** ci mostra il trend relativo ai reset della password e del nome dei profili monitorati da Sentinel.

Si nota che il numero di reset è costante a fine mese, plausibile considerando alcune policy di sicurezza aziendale. Nei giorni compresi tra il 14 e il 17 marzo invece c’è un attività sospetta. 

La query KQL relativa a questi dati è:
   ```
   SecurityEvent
   | where TimeGenerated > ago(100d)
   | where EventID in (4723, 4724, 4781)
   | summarize Count=count() by bin(TimeGenerated, 1h)
   ```

In [0]:
%sql

CREATE VIEW count_password_reset
AS( select DateTime, count(DateTime) as ResetCount
    from resetpassword
    where EventID in (4723, 4724, 4781)
    group by DateTime )

In [0]:
%sql
select Date, ResetCount
from plaindate left join count_password_reset on plaindate.Date = count_password_reset.DateTime
where Date > DATE'2021-12-28'



##### Considerazioni
    Questa attività potrebbe essere sintomo di tentativo di movimento laterale da parte di un attaccante.
    Quest’ultimo infatti cercherà di rendere inaccessibili i profili utente di cui ha appena ottenuto le credenziali di accesso.

    Le possibili azioni sono:
        - Stabilire regole più rigorose sulla generazione delle credenziali.
        - Sensibilizzare i dipendenti su tematiche phishing ed esposizione delle credenziali.

***

### 6. Rapporto tra tentativi di login totali e falliti

Nel nostro primo **indice di conformità** viene rappresentato un **grafico a dispersione** che mostra una relazione lineare tra il numero tentativi di login totali e i tentativi di login falliti. 

All’aumentare dell’uno è visibile un aumento dell’altro.

Sono presenti alcuni punti nel grafico in cui è presente un numero di login falliti sospetto. 

La query KQL relativa a quest grafico è:
   ```
SigninLogs
| where TimeGenerated > ago(100d)
| summarize Count_Tot=count() by Tot=bin(TimeGenerated, 1d)
| join kind=inner(
SigninLogs
| where TimeGenerated > ago(100d)
| where ResultType != 0
| summarize Count_Fail=count() by bin(TimeGenerated, 1d)) on $left.Tot==$right.TimeGenerated
| project-away Tot, TimeGenerated
| order by Count_Tot asc
   ```

In [0]:
%sql

CREATE OR REPLACE VIEW count_failed_login
AS( select Date, count(Date) as CountFailed
    from linearregression
    where ResultType != 0
    group by Date )

In [0]:
%sql
select *
from linearregression
order by Count_Tot

##### Considerazioni

    Queste anomalie potrebbero essere causate da un dictionary attack dove si cerca di avere accesso al sistema tramite brute force.

    Le possibili azioni sono:
        - Indagare sulla causa del picco di tentativi falliti.
        - Assicurarsi che l’attacco non abbia avuto esito positivo.


***

### 7. Controllo sull'integrità delle e-mail

Un ulteriore **indice di conformità** viene rappresentato tramite un grafico ad anello che mostra qual è la destinazione delle e-mail in arrivo sulle caselle postali istituzionali.
 

I risultati sono calcolati sulla base dell’azione finale intrapresa su un e-mail. Può derivare dal verdetto del filtro, dalle policies in vigore o dall’azione dell’utente.
 

La query KQL relativa a quest grafico è:
   ```
EmailEvents
| where TimeGenerated > ago(100d)
| where isnotempty(EmailAction)
| summarize Count=count() by EmailAction
   ```

In [0]:
%sql 
select EmailAction, count(*) as Count
from emailevents
group by EmailAction

##### Considerazioni

    Si può notare che circa il 22% viene mandato tra le junk mail mentre solo il 10% viene effettivamente riconosciuto come pericoloso e mandato in quarantena.
    
    Potrebbero essere presenti delle e-mail pericolose non trattate come tali.


    Le azioni a riguardo possono essere:
        - Rivisitazione dei criteri del filtro per e-mail
        - Sensibilizzazione dei dipendenti sul phishing.


***

### 8. Stato di protezione dei dispositivi

Questo ultimo indice è un **indice di conformità** e mostra il totale dei dispositivi monitorati dall’ambiente Sentinel, li suddivide in base al sistema operativo e ne mostra lo stato di protezione. 

Questo stato di protezione è dato dal livello di aggiornamento del dispositivo. Si può considerare un dispositivo protetto come un dispositivo correttamente configurato e aggiornato.

La query KQL relativa a quest grafico è:
```
ProtectionStatus
| where TimeGenerated > ago(100d)
| summarize Out_of_date=dcount(SourceComputerId) by ProtectionStatus, OSName, TypeofProtection
| where ProtectionStatus has "out of date"
| project OSName, Out_of_date
| union (
  ProtectionStatus
  | where TimeGenerated > ago(100d)
  | summarize Real_time_protection=dcount(SourceComputerId) by ProtectionStatus, OSName, TypeofProtection
  | where ProtectionStatus == "Real time protection" 
  | project OSName, Real_time_protection)
| union (
  ProtectionStatus
  | where TimeGenerated > ago(100d)
  | summarize Unprotected=dcount(SourceComputerId) by ProtectionStatus, OSName, TypeofProtection
  | where ProtectionStatus == "Unprotected / Unknown" 
  | project OSName, Unprotected)
```

In [0]:
%sql
select ProtectionStatus, OSName, count(*) as Count
from protectionstatus
group by ProtectionStatus, OSName


##### Considerazioni

    Il grafico mostra l’ecosistema di dispositivi presenta un buon livello di configurazione e aggiornamento sul lato server. 
    Lato endpoint invece sono presenti più dispositivi non protetti che possono presentare vulnerabilità che aumentano, per un utente malintenzionato, la superficie di attacco.
    
    Una soluzione potrebbe essere rendere più rigide le policies aziendali riguardo al patching (aggiornamento) dei dispositivi

***

### Fine

Grazie a tutti dell'attenzione.

![Horizon logo](assets/logo_horizon.png "Horizon Security s.r.l.")